# Create an initial dataset with elements from COCO dataset 
# and elements from densepose dataset condensing left-right in only one class
## only lower body parts included


# Getting data and looking into the structure

In [1]:
import cv2
import json
from json import JSONEncoder
import pandas as pd
import numpy as np
import pycocotools.mask as mask_util

from tqdm import tqdm

In [2]:
# version number of the json files
version = 1
# datapath = '/home/ubuntu/dormakaba/coco'
datapath = '/mnt/DATASSD/DataSandbox/coco'

In [3]:
with open(f'{datapath}/annotations/instances_train2014.json') as json_file:
    data = json.load(json_file)

In [4]:
with open(f'{datapath}/annotations/densepose_coco_2014_train.json') as json_file:
    person_data = json.load(json_file)

In [5]:
data_df = pd.json_normalize(data)
data_df.keys()

Index(['images', 'licenses', 'annotations', 'categories', 'info.description',
       'info.url', 'info.version', 'info.year', 'info.contributor',
       'info.date_created'],
      dtype='object')

In [6]:
person_data_df = pd.json_normalize(person_data)
person_data_df.keys()

Index(['images', 'annotations', 'categories'], dtype='object')

In [7]:
person_data_df['categories'].iloc[0][0]

{'supercategory': 'person',
 'id': 1,
 'name': 'person',
 'keypoints': ['nose',
  'left_eye',
  'right_eye',
  'left_ear',
  'right_ear',
  'left_shoulder',
  'right_shoulder',
  'left_elbow',
  'right_elbow',
  'left_wrist',
  'right_wrist',
  'left_hip',
  'right_hip',
  'left_knee',
  'right_knee',
  'left_ankle',
  'right_ankle'],
 'skeleton': [[16, 14],
  [14, 12],
  [17, 15],
  [15, 13],
  [12, 13],
  [6, 12],
  [7, 13],
  [6, 7],
  [6, 8],
  [7, 9],
  [8, 10],
  [9, 11],
  [2, 3],
  [1, 2],
  [1, 3],
  [2, 4],
  [3, 5],
  [4, 6],
  [5, 7]]}

In [8]:
data_df['categories'][0]

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [9]:
person_data_df['annotations'].iloc[0][0]

{'segmentation': [[453,
   292.1,
   457,
   253.1,
   439,
   245.1,
   438,
   215.1,
   439,
   198.1,
   420,
   223.1,
   414,
   233.1,
   401,
   227.1,
   400,
   226.1,
   398,
   229.1,
   391,
   231.1,
   387,
   213.1,
   399,
   203.1,
   404,
   200.1,
   413,
   194.1,
   418,
   186.1,
   408,
   181.1,
   415,
   154.1,
   418,
   142.1,
   419,
   127.1,
   422,
   125.1,
   419,
   120.1,
   412,
   122.1,
   407,
   112.1,
   402,
   105.1,
   389,
   113.1,
   390,
   105.1,
   395,
   100.1,
   395,
   97.1,
   398,
   83.1,
   407,
   72.1,
   417,
   71.1,
   424,
   72.1,
   428,
   73.1,
   436,
   80.1,
   441,
   90.1,
   446,
   96.1,
   456,
   101.1,
   472,
   110.1,
   480,
   113.1,
   493,
   123.1,
   499,
   136.1,
   504,
   147.1,
   509,
   167.1,
   515,
   182.1,
   531,
   205.1,
   532,
   218.1,
   525,
   229.1,
   514,
   246.1,
   499,
   283.1,
   499,
   307.1,
   499,
   323.1,
   499,
   343.1,
   505,
   367.1,
   505,
   380.1,
   

In [10]:
data_df['annotations'].iloc[0][0]

{'segmentation': [[312.29,
   562.89,
   402.25,
   511.49,
   400.96,
   425.38,
   398.39,
   372.69,
   388.11,
   332.85,
   318.71,
   325.14,
   295.58,
   305.86,
   269.88,
   314.86,
   258.31,
   337.99,
   217.19,
   321.29,
   182.49,
   343.13,
   141.37,
   348.27,
   132.37,
   358.55,
   159.36,
   377.83,
   116.95,
   421.53,
   167.07,
   499.92,
   232.61,
   560.32,
   300.72,
   571.89]],
 'area': 54652.9556,
 'iscrowd': 0,
 'image_id': 480023,
 'bbox': [116.95, 305.86, 285.3, 266.03],
 'category_id': 58,
 'id': 86}

# Generating classes

In [11]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                'scissors', 'teddy bear', 'hair drier', 'toothbrush']
                
# THE NEW CUSTOM CLASSES CONDENSE THE LEFT-RIGHT IN ONLY ONE CAT
CUSTOM_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear',
                'torso', 'hand', 'foot', 'upleg', 'lowleg', 'uparm', 'lowarm', 'head']

# THE NEW SMALL CUSTOM CLASSES CONDENSE THE LEFT-RIGHT IN ONLY ONE CAT
SMALL_CUSTOM_CLASSES = ['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 
                        'foot', 'upleg', 'lowleg']

CATEGORIES = [{'supercategory': 'person', 'id': 1, 'name': 'person'},
            {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
            {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
            {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
            {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
            {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
            {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
            {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
            {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
            {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
            {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
            {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
            {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
            {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
            {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
            {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
            {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
            {'supercategory': 'animal', 'id': 19, 'name': 'horse'},
            {'supercategory': 'animal', 'id': 20, 'name': 'sheep'},
            {'supercategory': 'animal', 'id': 21, 'name': 'cow'},
            {'supercategory': 'animal', 'id': 22, 'name': 'elephant'},
            {'supercategory': 'animal', 'id': 23, 'name': 'bear'},
            {'supercategory': 'animal', 'id': 24, 'name': 'zebra'},
            {'supercategory': 'animal', 'id': 25, 'name': 'giraffe'},
            {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
            {'supercategory': 'accessory', 'id': 28, 'name': 'umbrella'},
            {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
            {'supercategory': 'accessory', 'id': 32, 'name': 'tie'},
            {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
            {'supercategory': 'sports', 'id': 34, 'name': 'frisbee'},
            {'supercategory': 'sports', 'id': 35, 'name': 'skis'},
            {'supercategory': 'sports', 'id': 36, 'name': 'snowboard'},
            {'supercategory': 'sports', 'id': 37, 'name': 'sports ball'},
            {'supercategory': 'sports', 'id': 38, 'name': 'kite'},
            {'supercategory': 'sports', 'id': 39, 'name': 'baseball bat'},
            {'supercategory': 'sports', 'id': 40, 'name': 'baseball glove'},
            {'supercategory': 'sports', 'id': 41, 'name': 'skateboard'},
            {'supercategory': 'sports', 'id': 42, 'name': 'surfboard'},
            {'supercategory': 'sports', 'id': 43, 'name': 'tennis racket'},
            {'supercategory': 'kitchen', 'id': 44, 'name': 'bottle'},
            {'supercategory': 'kitchen', 'id': 46, 'name': 'wine glass'},
            {'supercategory': 'kitchen', 'id': 47, 'name': 'cup'},
            {'supercategory': 'kitchen', 'id': 48, 'name': 'fork'},
            {'supercategory': 'kitchen', 'id': 49, 'name': 'knife'},
            {'supercategory': 'kitchen', 'id': 50, 'name': 'spoon'},
            {'supercategory': 'kitchen', 'id': 51, 'name': 'bowl'},
            {'supercategory': 'food', 'id': 52, 'name': 'banana'},
            {'supercategory': 'food', 'id': 53, 'name': 'apple'},
            {'supercategory': 'food', 'id': 54, 'name': 'sandwich'},
            {'supercategory': 'food', 'id': 55, 'name': 'orange'},
            {'supercategory': 'food', 'id': 56, 'name': 'broccoli'},
            {'supercategory': 'food', 'id': 57, 'name': 'carrot'},
            {'supercategory': 'food', 'id': 58, 'name': 'hot dog'},
            {'supercategory': 'food', 'id': 59, 'name': 'pizza'},
            {'supercategory': 'food', 'id': 60, 'name': 'donut'},
            {'supercategory': 'food', 'id': 61, 'name': 'cake'},
            {'supercategory': 'furniture', 'id': 62, 'name': 'chair'},
            {'supercategory': 'furniture', 'id': 63, 'name': 'couch'},
            {'supercategory': 'furniture', 'id': 64, 'name': 'potted plant'},
            {'supercategory': 'furniture', 'id': 65, 'name': 'bed'},
            {'supercategory': 'furniture', 'id': 67, 'name': 'dining table'},
            {'supercategory': 'furniture', 'id': 70, 'name': 'toilet'},
            {'supercategory': 'electronic', 'id': 72, 'name': 'tv'},
            {'supercategory': 'electronic', 'id': 73, 'name': 'laptop'},
            {'supercategory': 'electronic', 'id': 74, 'name': 'mouse'},
            {'supercategory': 'electronic', 'id': 75, 'name': 'remote'},
            {'supercategory': 'electronic', 'id': 76, 'name': 'keyboard'},
            {'supercategory': 'electronic', 'id': 77, 'name': 'cell phone'},
            {'supercategory': 'appliance', 'id': 78, 'name': 'microwave'},
            {'supercategory': 'appliance', 'id': 79, 'name': 'oven'},
            {'supercategory': 'appliance', 'id': 80, 'name': 'toaster'},
            {'supercategory': 'appliance', 'id': 81, 'name': 'sink'},
            {'supercategory': 'appliance', 'id': 82, 'name': 'refrigerator'},
            {'supercategory': 'indoor', 'id': 84, 'name': 'book'},
            {'supercategory': 'indoor', 'id': 85, 'name': 'clock'},
            {'supercategory': 'indoor', 'id': 86, 'name': 'vase'},
            {'supercategory': 'indoor', 'id': 87, 'name': 'scissors'},
            {'supercategory': 'indoor', 'id': 88, 'name': 'teddy bear'},
            {'supercategory': 'indoor', 'id': 89, 'name': 'hair drier'},
            {'supercategory': 'indoor', 'id': 90, 'name': 'toothbrush'}]

FROM_COCO_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear']

EXTENDED_DENSEPOSE_CLASSES = ['torso', 'hand1', 'hand2', 
                    'foot1', 'foot2', 'upleg1', 'upleg2', 
                    'lowleg1', 'lowleg2','uparm1', 'uparm2', 
                    'lowarm1','lowarm2', 'head']

DENSEPOSE_CLASSES = ['torso', 'hand',  
                        'foot', 'upleg', 
                        'lowleg', 'uparm', 
                        'lowarm', 'head']

# realates image ID with category
COCO_LABEL_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8,
                  9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
                  18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24,
                  27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32,
                  37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40,
                  46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48,
                  54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56,
                  62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64,
                  74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
                  82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}

# this map relates body part position to unique identifier (avoiding left-right problem)
PERSON_LABEL_MAP = {0:0, 1: 1, 2: 1, 
                    3: 2, 4: 2, 5: 3, 6: 3, 
                    7: 4, 8: 4, 9: 5, 10: 5, 
                    11: 6, 12: 6, 13: 7}

In [12]:
REVERSE_COCO_LABEL_MAP = {}
for key, value in COCO_LABEL_MAP.items():
    REVERSE_COCO_LABEL_MAP[value] = key

# working with categories

In [17]:
# creates the categories list from COCO selected classes
custom_categories = []
for cat_data in data_df['categories'][0]:
    if cat_data['name'] in SMALL_CUSTOM_CLASSES:
        custom_categories.append(cat_data)
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'}]

In [18]:
# Adds hupan parts selected classes
i = 91
for nc in DENSEPOSE_CLASSES:
    if nc in SMALL_CUSTOM_CLASSES:
        c = {'supercategory': 'person', 'id': i, 'name': nc}
        custom_categories.append(c)
        i +=1
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
 {'supercategory': 'person', 'id': 91, 'name': 'foot'},
 {'supercategory': 'person', 'id': 92, 'name': 'upleg'},
 {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

In [19]:
# this is the list of the new categories
print(custom_categories)
print()
print(len(SMALL_CUSTOM_CLASSES), "-", len(custom_categories))

cc_list = [0]*(len(SMALL_CUSTOM_CLASSES))
CUSTOM_LABEL_MAP = {}
CUSTOM_ID_MAP = {}

for i, cc in enumerate(custom_categories):
    CUSTOM_ID_MAP[i] = cc['id']
    CUSTOM_LABEL_MAP[i] = cc['name']
    cc_list[i] = cc['name']

# the list of the names of the categories
print(CUSTOM_ID_MAP)
print(CUSTOM_LABEL_MAP)
print(cc_list)

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'animal', 'id': 17, 'name': 'cat'}, {'supercategory': 'animal', 'id': 18, 'name': 'dog'}, {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'}, {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'}, {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'}, {'supercategory': 'person', 'id': 91, 'name': 'foot'}, {'supercategory': 'person', 'id': 92, 'name': 'upleg'}, {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

9 - 9
{0: 2, 1: 17, 2: 18, 3: 27, 4: 31, 5: 33, 6: 91, 7: 92, 8: 93}
{0: 'bicycle', 1: 'cat', 2: 'dog', 3: 'backpack', 4: 'handbag', 5: 'suitcase', 6: 'foot', 7: 'upleg', 8: 'lowleg'}
['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 'foot', 'upleg', 'lowleg']


In [20]:
coco_category_id_list = [0]*len(cc_list)
for cat in data_df['categories'][0]:
    if cat['name'] in cc_list:
        print(cat, cat['name'], cat['id'])
        pos = cc_list.index(cat['name'])
        coco_category_id_list[pos] = cat['id']

# this is the list of the COCO ids of the new categories (when exists)
print(coco_category_id_list)
print(cc_list)

{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'} bicycle 2
{'supercategory': 'animal', 'id': 17, 'name': 'cat'} cat 17
{'supercategory': 'animal', 'id': 18, 'name': 'dog'} dog 18
{'supercategory': 'accessory', 'id': 27, 'name': 'backpack'} backpack 27
{'supercategory': 'accessory', 'id': 31, 'name': 'handbag'} handbag 31
{'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'} suitcase 33
[2, 17, 18, 27, 31, 33, 0, 0, 0]
['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 'foot', 'upleg', 'lowleg']


In [21]:
# this variable has the coco category id in the position of the custom category

custom_category_coco_category = [0]*len(cc_list)
for i, cat_name in enumerate(COCO_CLASSES):
    if cat_name in cc_list:
        cc_pos = cc_list.index(cat_name)
        coco_cat_id = REVERSE_COCO_LABEL_MAP[i+1]
        custom_category_coco_category[cc_pos] = coco_cat_id
        # print(i, cat_name, cc_pos, coco_cat_id)

print(custom_category_coco_category)
print(REVERSE_COCO_LABEL_MAP)

[2, 17, 18, 27, 31, 33, 0, 0, 0]
{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 27, 26: 28, 27: 31, 28: 32, 29: 33, 30: 34, 31: 35, 32: 36, 33: 37, 34: 38, 35: 39, 36: 40, 37: 41, 38: 42, 39: 43, 40: 44, 41: 46, 42: 47, 43: 48, 44: 49, 45: 50, 46: 51, 47: 52, 48: 53, 49: 54, 50: 55, 51: 56, 52: 57, 53: 58, 54: 59, 55: 60, 56: 61, 57: 62, 58: 63, 59: 64, 60: 65, 61: 67, 62: 70, 63: 72, 64: 73, 65: 74, 66: 75, 67: 76, 68: 77, 69: 78, 70: 79, 71: 80, 72: 81, 73: 82, 74: 84, 75: 85, 76: 86, 77: 87, 78: 88, 79: 89, 80: 90}


# looking at images

In [22]:
new_person_images = person_data_df['images'][0]
print( len(new_person_images), new_person_images[0].keys())

26437 dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])


In [23]:
images_person_data_df = pd.DataFrame(person_data_df['images'][0])

In [24]:
images_person_data_df.head(2)

,license,file_name,coco_url,height,width,date_captured,flickr_url,id
0,2,COCO_train2014_000000262145.jpg,http://images.cocodataset.org/train2014/COCO_t...,427,640,2013-11-20 02:07:55,http://farm8.staticflickr.com/7187/6967031859_...,262145
1,2,COCO_train2014_000000393224.jpg,http://images.cocodataset.org/train2014/COCO_t...,640,427,2013-11-18 02:29:39,http://farm9.staticflickr.com/8503/8356977703_...,393224


In [25]:
person_data_df['images'][0][0]

{'license': 2,
 'file_name': 'COCO_train2014_000000262145.jpg',
 'coco_url': 'http://images.cocodataset.org/train2014/COCO_train2014_000000262145.jpg',
 'height': 427,
 'width': 640,
 'date_captured': '2013-11-20 02:07:55',
 'flickr_url': 'http://farm8.staticflickr.com/7187/6967031859_5f08387bde_z.jpg',
 'id': 262145}

# looking at images
## Those are the images I'll use in the dataset that come from 
## the DENSEPOSE dataset

In [26]:
# list of image ids from human parts data
# from here we will use all images
new_person_images_list = [a['id'] for a in person_data_df['images'][0]]
len(new_person_images_list)

26437

In [27]:
# list of img ids from coco dataset
# that have a cat id in the list of coco selected categories
# removing duplicates
all_images_list = set([a['image_id'] for a in data_df['annotations'][0] if a['category_id'] in coco_category_id_list])
len(all_images_list)

14964

In [28]:
images_from_coco_not_in_dense = [img for img in all_images_list if img not in new_person_images_list]
len(images_from_coco_not_in_dense)

9394

In [29]:
images_from_dense_not_in_custom = [img for img in new_person_images_list if img not in all_images_list]
len(images_from_dense_not_in_custom)

20867

In [30]:
len(data_df['images'][0])

82783

# annotation keys

In [31]:
# person annotations
# extraer segmentation: anotaciòn individual de cada persona detectada
# extrater dp_masks: e insertar una anotación por cada uno de los 14 elementos de la persona que tienen anotaciones
# si se puede, transformar el mapa de bits en 
print(person_data_df['annotations'][0][0].keys())
print(person_data_df['annotations'][0][0]['dp_masks'])

dict_keys(['segmentation', 'num_keypoints', 'dp_masks', 'area', 'dp_I', 'iscrowd', 'keypoints', 'dp_x', 'dp_U', 'image_id', 'dp_V', 'bbox', 'category_id', 'dp_y', 'id'])
[{'counts': '\\Qa03k7200M210N110N1N2100000O1000O0010O2O0O10001O0O1N200O2O001O0O1O?B0O:G0O;F017H5K011N001O010O2N001O010O002N001O1O011N001O010OO1000001O000O2O00000000001O00001O001N1000001O0O1000001N101O00001N101O002N1O002N002N002M102N1N103M001O002M2O003M001N102N2N002N001O002M3N002N002M101O002N2M102N001O002N2N002M102N001OUU9', 'size': [256, 256]}, {'counts': 'lk5110l70TH0j7400O2O0NNZHNf72ZHNe77O01003M0O010O10O100000O10000O01000000O10000O10O1001O001N101M3N101M20[\\_1', 'size': [256, 256]}, {'counts': 'i33m70L5O0O2O0O100O1OL`HIa77_HI_79_HIa77_HI`77`HK_75aHK_7:1N20OO200000000000O1000O10O0101N2N101O001N102M`Tf1', 'size': [256, 256]}, [], {'counts': 'e_Y12m73N0O2O002N0O10000000000000000000000000000000M2L50_P`0', 'size': [256, 256]}, {'counts': 'im[14l70N110L400M3N200M300M300M3N200M300L400N2M4O0M@c00ON30NN40ON0120ON30NO30NN3O20

In [32]:
print(person_data_df['annotations'][0][0])

{'segmentation': [[453, 292.1, 457, 253.1, 439, 245.1, 438, 215.1, 439, 198.1, 420, 223.1, 414, 233.1, 401, 227.1, 400, 226.1, 398, 229.1, 391, 231.1, 387, 213.1, 399, 203.1, 404, 200.1, 413, 194.1, 418, 186.1, 408, 181.1, 415, 154.1, 418, 142.1, 419, 127.1, 422, 125.1, 419, 120.1, 412, 122.1, 407, 112.1, 402, 105.1, 389, 113.1, 390, 105.1, 395, 100.1, 395, 97.1, 398, 83.1, 407, 72.1, 417, 71.1, 424, 72.1, 428, 73.1, 436, 80.1, 441, 90.1, 446, 96.1, 456, 101.1, 472, 110.1, 480, 113.1, 493, 123.1, 499, 136.1, 504, 147.1, 509, 167.1, 515, 182.1, 531, 205.1, 532, 218.1, 525, 229.1, 514, 246.1, 499, 283.1, 499, 307.1, 499, 323.1, 499, 343.1, 505, 367.1, 505, 380.1, 505, 381.1, 486, 387.1, 482, 392.1, 479, 393.1, 469, 363.1, 453, 343.1, 451, 339.1, 454, 321.1, 453, 312.1, 460, 313.1, 458, 298.1, 452, 293.1]], 'num_keypoints': 15, 'dp_masks': [{'counts': '\\Qa03k7200M210N110N1N2100000O1000O0010O2O0O10001O0O1N200O2O001O0O1O?B0O:G0O;F017H5K011N001O010O2N001O010O002N001O1O011N001O010OO1000001O0

# generating annotations

In [33]:
def GetDensePoseMasks(Polys):
    mask_list = [0]*15
    for i in range(1,15):
        MaskGen = np.zeros([256,256])
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = 1
        mask_list[i-1]=MaskGen
    return mask_list

In [34]:
def GetDensePoseMask(encoded_mask):
    MaskGen = np.zeros([256,256])
    current_mask = mask_util.decode(encoded_mask)
    MaskGen[current_mask>0] = 1
    return MaskGen

In [35]:
def get_poligons(mask):
    c = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [0]*2*len(c[0][0].tolist())
    for i,x in enumerate(c[0][0].tolist()):
        contours[2*i]=int(x[0][0])
        contours[2*i+1]= int(x[0][1])
    return contours

In [36]:
def CreateMaskBBox(mask, bbr, image_shape):
    final_mask = np.zeros([image_shape[1],image_shape[0]])
    x1 = int(bbr[0])
    y1 = int(bbr[1])
    x2 = int(bbr[0]+bbr[2])
    y2 = int(bbr[1]+bbr[3])
    x2 = min( [ x2, image_shape[0] ] )
    y2 = min( [ y2, image_shape[1] ] )
    # print(bbr, (x1,y1,x2,y2), (int(x2-x1),int(y2-y1)), mask.shape)
    resized_mask = cv2.resize( mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)
    final_mask[y1:y2,x1:x2] = resized_mask
    az = np.nonzero(final_mask)
    bby1,bby2,bbx1,bbx2=np.min(az[0]), np.max(az[0]), np.min(az[1]), np.max(az[1])
    contours = get_poligons(final_mask.astype(np.uint8))
    encoded_final_mask = mask_util.encode(np.asfortranarray(final_mask.astype(np.uint8)))
    mask_area = int(mask_util.area(encoded_final_mask))
    # print((bbx1, bby1, bbx2-bbx1, bby2-bby1),encoded_final_mask)
    encoded_final_mask['counts'] = encoded_final_mask['counts'].decode("utf-8")
    return final_mask, (bbx1, bby1, bbx2-bbx1, bby2-bby1), mask_area, encoded_final_mask, contours

# create new annotations in coco style suitable to train YOLACT model

In [37]:
new_person_ann = []
for annotation in tqdm(person_data_df['annotations'][0]):
    # print(annotation['image_id'], annotation['bbox'])
    if annotation['iscrowd']==0 and annotation['image_id'] in new_person_images_list:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_cat_id = 1 # only person cat so must be 1
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': 0}
        new_person_ann.append(new_data)
        a = images_person_data_df[images_person_data_df['id']==ann_image_id]
        image_size = [a['width'].values[0],a['height'].values[0]]

        if 'dp_masks' in annotation:
            for i, ann_mask in enumerate(annotation['dp_masks']):
                if len(ann_mask) > 1:
                    ann_mask_decoded = GetDensePoseMask(ann_mask)
                    mask, bbox, ann_area, encoded_mask, contours = CreateMaskBBox(ann_mask_decoded, annotation['bbox'], image_size )
                    ann_bbox = [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                    ann_segment = encoded_mask
                    ann_image_id = annotation['image_id']
                    ann_id = -1
                    # for reusing ids I'll prepare a first approach 
                    # with a cat id dictionary
                    ann_cat_id = 91+PERSON_LABEL_MAP[i]
                    if len(contours)>4:
                        new_data = {'segmentation': [contours], # encoded_mask,
                                    'area': ann_area,
                                    'image_id': ann_image_id,
                                    'id': ann_id,
                                    'category_id': ann_cat_id,
                                    'bbox': ann_bbox,
                                    'iscrowd': 0}
                        new_person_ann.append(new_data)

100%|██████████| 100403/100403 [21:29<00:00, 77.84it/s]


In [38]:
xxx1 = [x for x in new_person_ann if x['category_id']==1]
xxx2 = [x for x in new_person_ann if x['category_id']!=1]
print(len(xxx1),len(xxx2))

98816 375482


In [39]:
# check that the list of annotated image_ids has the len len as densepose image id's
new_person_images_annotated_list = set([a['image_id'] for a in new_person_ann])
print( len(new_person_images_annotated_list), len(set(new_person_images_annotated_list)), len(set(new_person_images_list)))

26437 26437 26437


# till here I have all human data and parts segmented and annotated
# create new annotations from COCO dataset for the selected images

In [40]:
person_ann_list1 = [x for x in data_df['annotations'][0] if x['image_id'] in all_images_list and x['image_id'] not in new_person_images_list and x['category_id']==1 and x['iscrowd']==0]
person_ann_list2 = [x for x in data_df['annotations'][0] if x['image_id'] not in new_person_images_list and x['category_id'] in custom_category_coco_category and x['iscrowd']==0]
person_ann_list3 = [x for x in data_df['annotations'][0] if x['image_id'] in new_person_images_list and x['category_id']==1 and x['iscrowd']==0]
print(len(data_df['annotations'][0]), len(person_ann_list1), len(person_ann_list2) , len(person_ann_list3))

604907 25718 19056 98816


In [41]:
person_ann_list1[0]


{'segmentation': [[329.88,
   211.23,
   337.63,
   211.93,
   341.51,
   210.17,
   341.86,
   206.65,
   340.1,
   202.77,
   335.52,
   199.6,
   335.52,
   197.13,
   335.52,
   190.44,
   339.04,
   186.21,
   342.57,
   170.35,
   345.74,
   159.43,
   342.57,
   154.14,
   340.1,
   148.86,
   336.93,
   143.92,
   337.28,
   138.28,
   344.68,
   138.28,
   340.45,
   122.43,
   339.75,
   116.79,
   334.46,
   107.63,
   320.37,
   104.1,
   325.65,
   97.06,
   326.0,
   91.06,
   321.77,
   82.25,
   316.49,
   89.66,
   305.21,
   87.19,
   304.51,
   92.47,
   304.16,
   97.06,
   305.21,
   100.58,
   304.16,
   103.4,
   292.53,
   105.51,
   284.42,
   123.84,
   278.43,
   140.05,
   273.14,
   146.74,
   273.85,
   152.73,
   279.49,
   156.26,
   281.6,
   154.14,
   284.07,
   144.63,
   289.0,
   133.0,
   293.23,
   133.7,
   294.99,
   125.95,
   295.35,
   157.67,
   312.26,
   157.31,
   318.96,
   150.27,
   335.17,
   150.27,
   333.76,
   157.31,
   324.95,


In [42]:
537548 in new_person_images_list


False

In [43]:
new_data_ann = []
new_data_image_list = []
new_data_images = []
for annotation in tqdm(data_df['annotations'][0]):
    if annotation['category_id'] == 1:
        if annotation['image_id'] not in new_person_images_list:
            # if annotation is person and is not in human parts annotation
            ann_segment = annotation['segmentation']
            ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
            ann_area = annotation['area']
            ann_image_id = annotation['image_id']
            ann_id = annotation['id']
            ann_iscrowd = annotation['iscrowd']
            ann_cat_id = annotation['category_id']
            new_data = {'segmentation': ann_segment,
                        'area': ann_area,
                        'image_id': ann_image_id,
                        'id': ann_id,
                        'category_id': ann_cat_id,
                        'bbox': ann_bbox,
                        'iscrowd': ann_iscrowd}
            new_data_ann.append(new_data)
            new_data_image_list.append(ann_image_id)
    elif annotation['category_id'] in custom_category_coco_category:
        # if annotation is not person
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_iscrowd = annotation['iscrowd']
        ann_cat_id = annotation['category_id']
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': ann_iscrowd}
        new_data_ann.append(new_data)
        new_data_image_list.append(ann_image_id)

100%|██████████| 604907/604907 [01:27<00:00, 6892.86it/s]


In [44]:
print(len(new_data_ann), len(set(new_data_image_list)))

116172 29260


In [45]:
new_data_images = [image for image in data_df['images'][0] if image['id'] in set(new_data_image_list)]

# take a look to "custom categories"

In [46]:
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
 {'supercategory': 'person', 'id': 91, 'name': 'foot'},
 {'supercategory': 'person', 'id': 92, 'name': 'upleg'},
 {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

# Saving humans parts data

In [47]:

new_human_part_data = {
    'images': new_person_images, 
    'annotations': new_person_ann, 
    'categories': custom_categories
}

In [48]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_human_part_data_json = MyEncoder().encode(new_human_part_data)

In [49]:
with open(f'{datapath}/annotations/custom_small_ann_human_parts_train2014_v{version}.json', 'w') as f:
    json.dump(new_human_part_data_json, f)

# Saving non humans data


In [50]:

new_non_human_data = {
    'images': new_data_images, 
    'annotations': new_data_ann, 
    'categories': custom_categories
}

In [51]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_non_human_data_json = MyEncoder().encode(new_non_human_data)

In [52]:
with open(f'{datapath}/annotations/custom_small_ann_non_human_train2014_v{version}.json', 'w') as f:
    json.dump(new_non_human_data_json, f)

# updating indices

In [53]:
final_images = new_person_images + new_data_images
final_annotations = new_person_ann + new_data_ann

In [61]:
nil = []
for image in tqdm(final_images):
    try:
        if image["id"] not in all_images_list:
            nil.append(image["id"])
    except Exception as e:
        #print(image)
        pass
len(nil)

100%|██████████| 55697/55697 [00:00<00:00, 1868343.12it/s]


35163

In [56]:
final_annotations[0].keys()

dict_keys(['segmentation', 'area', 'image_id', 'id', 'category_id', 'bbox', 'iscrowd'])

In [57]:
for i in tqdm(range(len(final_annotations))):
    final_annotations[i]['id'] = i+1

100%|██████████| 590470/590470 [00:00<00:00, 2733635.27it/s]


In [62]:
nil = []
for image in tqdm(final_images):
    try:
        if image["id"] not in all_images_list:
            nil.append(image["id"])
    except Exception as e:
        #print(image)
        pass
len(nil)

100%|██████████| 55697/55697 [00:00<00:00, 2503323.51it/s]


35163

# saving all data

In [63]:
# IN THE version_02
new_person_data = {
    'images': final_images, 
    'annotations': final_annotations, 
    'categories': custom_categories
}

In [64]:
from json import JSONEncoder
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_person_data_json = MyEncoder().encode(new_person_data)

In [65]:
with open(f'{datapath}/annotations/custom_small_ann_train2014_v{version}.json', 'w') as f:
    json.dump(new_person_data_json, f)